# Practical box size estimation from RFID interrogation traces using transfer learning 

In [19]:
# Bibliotecas necesarias
import numpy as np
import random
import os

# Configuramos matplotlib
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Semilla fija para reproducibilidad
np.random.seed(1)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [20]:
# Entrenamos un autoencoder construido en keras
# Incluimos dos callbacks para ir guardando el modelo y para parar por 
# earlystopping

import keras

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf


In [23]:
import pandas

index = pandas.read_csv("IMP/index.IMP.txt")
index["numframes"] = index.framelst-index.frameini+1
maxframes = index.numframes.max()
print(f'maxframes: {maxframes}')
frames = pandas.read_csv("IMP/frames.IMP.txt")
frames.head(-1)

maxframes: 29


,tagsread,pwread,pwcol,pwmin,pwmax
0,9,0.559926,0.275668,0.153588,0.709296
1,8,0.594366,0.270182,0.277252,0.727058
2,10,0.420491,0.236543,0.146248,0.634058
3,10,0.324143,0.203427,0.061934,0.550386
4,12,0.316759,0.143247,0.141165,0.497909
...,...,...,...,...,...
115844,9,0.594776,0.476232,0.520391,0.713048
115845,9,0.566135,0.418666,0.365887,0.671997
115846,11,0.460723,0.412010,0.220976,0.710323
115847,9,0.453314,0.354898,0.297078,0.633767


In [24]:
index.head(-1)

,frameini,framelst,boxnum,boxoriginal,L,W,H,read,tags,tagsread,readtime,despX,despY,theta,vhr,ha,hb,d,numframes
0,0,9,3,3,1.120,1.000,0.920,1,84,84,0.21152,0,0,0,1,2.5,1,2.5,10
1,10,10,0,0,0.236,0.196,0.172,1,1,1,0.00598,0,0,0,1,2.5,1,2.5,1
2,11,14,2,2,1.075,0.870,0.550,1,38,38,0.09963,0,0,0,1,2.5,1,2.5,4
3,15,33,3,3,1.120,1.000,0.920,1,121,121,0.36487,0,0,0,1,2.5,1,2.5,19
4,34,45,3,3,1.120,1.000,0.920,1,101,101,0.25069,0,0,0,1,2.5,1,2.5,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,115806,115808,1,1,0.762,0.508,0.510,1,15,15,0.04638,0,0,0,1,2.5,1,2.5,3
19995,115809,115822,3,3,1.120,1.000,0.920,1,108,108,0.28904,0,0,0,1,2.5,1,2.5,14
19996,115823,115826,1,1,0.762,0.508,0.510,1,25,25,0.07453,0,0,0,1,2.5,1,2.5,4
19997,115827,115836,3,3,1.120,1.000,0.920,1,89,89,0.22128,0,0,0,1,2.5,1,2.5,10


In [30]:
import pickle
picklefile = f'data.pickle'
# %system rm -f data.pickle

try: 
    with open(picklefile, 'rb') as f:
        X, t = pickle.load(f)
        print(f'Datos ya disponibles de {picklefile}')
except:
    SAMPLES = 19999
    X = np.zeros(shape=[SAMPLES, 23]) 
    t = np.zeros(shape=[SAMPLES, ]) 
    print(f'X.shape: {X.shape}')

    for i in range(SAMPLES):
        batch = random.randint(0,X.shape[0]) 
        tagsread = index.iloc[batch].tagsread
        readtime = index.iloc[batch].readtime
        numframes = index.iloc[batch].numframes
        framebase = int(index.iloc[batch].frameini) 
        aux = frames.loc[framebase:framebase+numframes-1].values
        X[i,0] = tagsread
        X[i,1] = readtime
        X[i,2] = numframes
        X[i,3:8] = aux.mean(axis=0)
        X[i,8:13] = aux.std(axis=0)
        X[i,13:18] = aux.min(axis=0)
        X[i,18:23] = aux.max(axis=0)
        t[i] = index.iloc[batch].boxoriginal
        #X[batch,3:3+numframes*5] = frames.loc[framebase:framebase+numframes-1].values.ravel()
        if (i+1)%5000==0:
            print(f'i: {i+1}, batch: {batch}, frames: {numframes}, framebase: {framebase}, t: {t[i]}, aux: {aux}')
        
    t = t.astype(int)
    with open(picklefile, 'wb') as f:
        pickle.dump([X, t], f)

X.shape: (19999, 23)
i: 5000, batch: 14835, frames: 3.0, framebase: 85756, t: 1.0, aux: [[12.          0.61596685  0.61149299  0.45816848  0.70507012]
 [ 7.          0.57415963  0.          0.52757476  0.67614019]
 [ 2.          0.49808366  0.          0.46735813  0.52880918]]
i: 10000, batch: 2248, frames: 7.0, framebase: 13283, t: 2.0, aux: [[10.          0.48266997  0.47081853  0.36106278  0.63101498]
 [11.          0.54187794  0.42657241  0.35706067  0.71388698]
 [11.          0.50772452  0.4048393   0.24956105  0.63636345]
 [11.          0.38172581  0.39907314  0.182518    0.55819448]
 [ 8.          0.35136134  0.39232464  0.21558325  0.56687598]
 [ 3.          0.38422007  0.27790219  0.29197261  0.47720366]
 [ 2.          0.27790219  0.          0.23951807  0.31628632]]
i: 15000, batch: 10785, frames: 2.0, framebase: 62602, t: 1.0, aux: [[12.          0.59244969  0.          0.44226882  0.70335441]
 [ 6.          0.57149286  0.          0.43495517  0.68155938]]


In [31]:
def change_distro(X,t,samples):
    
    newX = np.zeros(shape=[samples.sum(),X.shape[1]])
    newt = np.zeros(shape=[samples.sum(),])
    numsamples = 0
    for i in range(t.max()+1):
        aux = X[t==i]
        newX[numsamples:numsamples+samples[i]] = aux[:samples[i]]
        newt[numsamples:numsamples+samples[i]] = i
        numsamples += samples[i]
    
    newt = newt.astype(int)
    return newX,newt

In [33]:
X,t = change_distro(X,t,(1500*np.ones(11,).astype(int)))
print(X.shape, t.shape)

(16500, 23) (16500,)


In [34]:
with np.printoptions(precision=3, suppress=True):
    print(X[0:5,:])
    
t = t.astype(int)
print(t.min())
print(t.max())

a = np.sum(X)
j = np.isnan(a)
print(a,j)

for i in range(X.shape[0]):
    a = np.sum(X[i,:])
    if np.isnan(a): 
        print(f'Nan en {i}: {X[i]}')

[[1.    0.006 1.    1.    0.744 0.    0.744 0.744 0.    0.    0.    0.
  0.    1.    0.744 0.    0.744 0.744 1.    0.744 0.    0.744 0.744]
 [1.    0.006 1.    1.    0.748 0.    0.748 0.748 0.    0.    0.    0.
  0.    1.    0.748 0.    0.748 0.748 1.    0.748 0.    0.748 0.748]
 [1.    0.006 1.    1.    0.715 0.    0.715 0.715 0.    0.    0.    0.
  0.    1.    0.715 0.    0.715 0.715 1.    0.715 0.    0.715 0.715]
 [1.    0.006 1.    1.    0.744 0.    0.744 0.744 0.    0.    0.    0.
  0.    1.    0.744 0.    0.744 0.744 1.    0.744 0.    0.744 0.744]
 [2.    0.008 1.    2.    0.746 0.    0.736 0.756 0.    0.    0.    0.
  0.    2.    0.746 0.    0.736 0.756 2.    0.746 0.    0.736 0.756]]
0
3
450577.2566650511 False


In [35]:
import keras

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

from sklearn.model_selection import train_test_split
X_train, X_test, t_train, t_test =  train_test_split(X, t, test_size=0.2)

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

standard_transformer = Pipeline(steps=[
        ('standard', StandardScaler())])

minmax_transformer = Pipeline(steps=[
        ('minmax', MinMaxScaler())])

preprocessor = ColumnTransformer(
        remainder='passthrough', #passthough features not listed
        transformers=[
            ('std', standard_transformer , []),#slice(0,X.shape[1])),
            ('mm', minmax_transformer , slice(0,X.shape[1]))
        ])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

with np.printoptions(precision=3, suppress=True):
    print(X_train[:2],t_train[:2],'\n',X_test.shape)

[[0.286 0.241 0.207 0.609 0.522 0.432 0.326 0.664 0.749 0.27  0.594 0.15
  0.451 0.1   0.348 0.    0.208 0.348 0.812 0.697 0.665 0.405 0.879]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]] [2 0] 
 (3300, 23)


In [36]:
model = keras.models.Sequential([
#    keras.layers.Dense(100, activation="selu", input_shape=X_train.shape[1:], kernel_regularizer=tf.keras.regularizers.l1(0.01)),
  #keras.layers.Dense(100, activation="relu", input_shape=X_train.shape[1:], kernel_regularizer=tf.keras.regularizers.l1(0.001)),
    keras.layers.Dense(100, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(100, activation="tanh"),
    keras.layers.Dense(50, activation="tanh"),
   # keras.layers.Dense(t.max()+1, activation="softmax")
    keras.layers.Dense(int(t.max())+1, activation="sigmoid")
   # keras.layers.Dense(3, activation="linear")
])
model.summary()

Model: "sequential_102"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_305 (Dense)            (None, 100)               2400      
_________________________________________________________________
dense_306 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_307 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_308 (Dense)            (None, 4)                 204       
Total params: 17,754
Trainable params: 17,754
Non-trainable params: 0
_________________________________________________________________


In [ ]:
modelname = "model.h5"

try: 
    model = keras.models.load_model(modelname) # cargar modelo
except:
    pass

logdir = "./log"
early_stopping_cb = keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint(modelname, save_best_only=True)
tb_cb = keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
#model.compile(loss="mse", optimizer="adam", metrics=["accuracy"])
history = model.fit(X_train, t_train, epochs=50000, 
                    validation_data=(X_test, t_test), 
                    batch_size=32,
                    callbacks=[checkpoint_cb, early_stopping_cb])

Train on 13200 samples, validate on 3300 samples
Epoch 1/50000
13200/13200 [==============================] - 4s 323us/step - loss: 0.1645 - accuracy: 0.9725 - val_loss: 0.0093 - val_accuracy: 0.9991
Epoch 2/50000
13200/13200 [==============================] - 3s 210us/step - loss: 0.0049 - accuracy: 0.9996 - val_loss: 0.0039 - val_accuracy: 0.9997
Epoch 3/50000
13200/13200 [==============================] - 3s 203us/step - loss: 0.0026 - accuracy: 0.9997 - val_loss: 0.0032 - val_accuracy: 0.9997
Epoch 4/50000
13200/13200 [==============================] - 2s 171us/step - loss: 0.0030 - accuracy: 0.9995 - val_loss: 0.0027 - val_accuracy: 0.9997
Epoch 5/50000
13200/13200 [==============================] - 3s 201us/step - loss: 0.0019 - accuracy: 0.9995 - val_loss: 0.0022 - val_accuracy: 0.9997
Epoch 6/50000
13200/13200 [==============================] - 2s 173us/step - loss: 0.0014 - accuracy: 0.9997 - val_loss: 0.0011 - val_accuracy: 0.9997
Epoch 7/50000
13200/13200 [==================

Epoch 104/50000
13200/13200 [==============================] - 2s 185us/step - loss: 3.6840e-07 - accuracy: 1.0000 - val_loss: 4.8643e-07 - val_accuracy: 1.0000
Epoch 105/50000
13200/13200 [==============================] - 4s 291us/step - loss: 3.6576e-07 - accuracy: 1.0000 - val_loss: 3.8674e-07 - val_accuracy: 1.0000
Epoch 106/50000
13200/13200 [==============================] - 4s 269us/step - loss: 0.0107 - accuracy: 0.9989 - val_loss: 0.0046 - val_accuracy: 0.9997
Epoch 107/50000
13200/13200 [==============================] - 3s 212us/step - loss: 0.0029 - accuracy: 0.9995 - val_loss: 6.2668e-04 - val_accuracy: 0.9997
Epoch 108/50000
13200/13200 [==============================] - 3s 238us/step - loss: 7.8571e-04 - accuracy: 0.9998 - val_loss: 7.8956e-04 - val_accuracy: 0.9997
Epoch 109/50000
13200/13200 [==============================] - 3s 248us/step - loss: 3.6230e-05 - accuracy: 1.0000 - val_loss: 6.6732e-04 - val_accuracy: 0.9997
Epoch 110/50000
13200/13200 [=================

Epoch 155/50000
13200/13200 [==============================] - 3s 208us/step - loss: 3.6067e-07 - accuracy: 1.0000 - val_loss: 7.2692e-07 - val_accuracy: 1.0000
Epoch 156/50000
13200/13200 [==============================] - 2s 168us/step - loss: 3.6036e-07 - accuracy: 1.0000 - val_loss: 1.9037e-06 - val_accuracy: 1.0000
Epoch 157/50000
13200/13200 [==============================] - 2s 162us/step - loss: 3.6067e-07 - accuracy: 1.0000 - val_loss: 4.1676e-07 - val_accuracy: 1.0000
Epoch 158/50000
13200/13200 [==============================] - 2s 167us/step - loss: 3.5942e-07 - accuracy: 1.0000 - val_loss: 5.0826e-07 - val_accuracy: 1.0000
Epoch 159/50000
13200/13200 [==============================] - 2s 176us/step - loss: 3.5924e-07 - accuracy: 1.0000 - val_loss: 4.6060e-07 - val_accuracy: 1.0000
Epoch 160/50000
13200/13200 [==============================] - 2s 158us/step - loss: 3.5913e-07 - accuracy: 1.0000 - val_loss: 4.7977e-07 - val_accuracy: 1.0000
Epoch 161/50000
13200/13200 [=====

Epoch 206/50000
13200/13200 [==============================] - 2s 153us/step - loss: 3.5783e-07 - accuracy: 1.0000 - val_loss: 3.6236e-07 - val_accuracy: 1.0000
Epoch 207/50000
13200/13200 [==============================] - 2s 156us/step - loss: 3.5784e-07 - accuracy: 1.0000 - val_loss: 3.6142e-07 - val_accuracy: 1.0000
Epoch 208/50000
13200/13200 [==============================] - 2s 162us/step - loss: 3.5781e-07 - accuracy: 1.0000 - val_loss: 3.6214e-07 - val_accuracy: 1.0000
Epoch 209/50000
13200/13200 [==============================] - 3s 193us/step - loss: 3.5785e-07 - accuracy: 1.0000 - val_loss: 3.6146e-07 - val_accuracy: 1.0000
Epoch 210/50000
13200/13200 [==============================] - 2s 155us/step - loss: 3.5784e-07 - accuracy: 1.0000 - val_loss: 3.6146e-07 - val_accuracy: 1.0000
Epoch 211/50000
13200/13200 [==============================] - 2s 156us/step - loss: 3.5783e-07 - accuracy: 1.0000 - val_loss: 3.6211e-07 - val_accuracy: 1.0000
Epoch 212/50000
13200/13200 [=====

Epoch 257/50000
13200/13200 [==============================] - 3s 209us/step - loss: 3.5775e-07 - accuracy: 1.0000 - val_loss: 3.5943e-07 - val_accuracy: 1.0000
Epoch 258/50000
13200/13200 [==============================] - 2s 176us/step - loss: 3.5775e-07 - accuracy: 1.0000 - val_loss: 3.5947e-07 - val_accuracy: 1.0000
Epoch 259/50000
13200/13200 [==============================] - 3s 196us/step - loss: 3.5775e-07 - accuracy: 1.0000 - val_loss: 3.5958e-07 - val_accuracy: 1.0000
Epoch 260/50000
13200/13200 [==============================] - 2s 164us/step - loss: 3.5777e-07 - accuracy: 1.0000 - val_loss: 3.5958e-07 - val_accuracy: 1.0000
Epoch 261/50000
13200/13200 [==============================] - 3s 237us/step - loss: 3.5775e-07 - accuracy: 1.0000 - val_loss: 3.5980e-07 - val_accuracy: 1.0000
Epoch 262/50000
13200/13200 [==============================] - 2s 172us/step - loss: 3.5775e-07 - accuracy: 1.0000 - val_loss: 3.5969e-07 - val_accuracy: 1.0000
Epoch 263/50000
13200/13200 [=====

...

Epoch 512/50000
13200/13200 [==============================] - 2s 181us/step - loss: 3.5766e-07 - accuracy: 1.0000 - val_loss: 3.5817e-07 - val_accuracy: 1.0000
Epoch 513/50000
13200/13200 [==============================] - 2s 154us/step - loss: 3.5765e-07 - accuracy: 1.0000 - val_loss: 3.5821e-07 - val_accuracy: 1.0000
Epoch 514/50000
13200/13200 [==============================] - 2s 149us/step - loss: 3.5765e-07 - accuracy: 1.0000 - val_loss: 3.5824e-07 - val_accuracy: 1.0000
Epoch 515/50000
13200/13200 [==============================] - 2s 161us/step - loss: 3.5765e-07 - accuracy: 1.0000 - val_loss: 3.5817e-07 - val_accuracy: 1.0000
Epoch 516/50000
13200/13200 [==============================] - 2s 168us/step - loss: 3.5765e-07 - accuracy: 1.0000 - val_loss: 3.5821e-07 - val_accuracy: 1.0000
Epoch 517/50000
13200/13200 [==============================] - 2s 156us/step - loss: 3.5765e-07 - accuracy: 1.0000 - val_loss: 3.5831e-07 - val_accuracy: 1.0000
Epoch 518/50000
13200/13200 [=====

Epoch 563/50000
13200/13200 [==============================] - 2s 153us/step - loss: 3.5766e-07 - accuracy: 1.0000 - val_loss: 3.5810e-07 - val_accuracy: 1.0000
Epoch 564/50000
13200/13200 [==============================] - 2s 178us/step - loss: 3.5765e-07 - accuracy: 1.0000 - val_loss: 3.5810e-07 - val_accuracy: 1.0000
Epoch 565/50000
13200/13200 [==============================] - 2s 153us/step - loss: 3.5765e-07 - accuracy: 1.0000 - val_loss: 3.5803e-07 - val_accuracy: 1.0000
Epoch 566/50000
13200/13200 [==============================] - 2s 164us/step - loss: 3.5767e-07 - accuracy: 1.0000 - val_loss: 3.5810e-07 - val_accuracy: 1.0000
Epoch 567/50000
13200/13200 [==============================] - 2s 153us/step - loss: 3.5765e-07 - accuracy: 1.0000 - val_loss: 3.5810e-07 - val_accuracy: 1.0000
Epoch 568/50000
13200/13200 [==============================] - 2s 156us/step - loss: 3.5767e-07 - accuracy: 1.0000 - val_loss: 3.5813e-07 - val_accuracy: 1.0000
Epoch 569/50000
13200/13200 [=====

Epoch 614/50000
13200/13200 [==============================] - 2s 175us/step - loss: 3.5766e-07 - accuracy: 1.0000 - val_loss: 3.5799e-07 - val_accuracy: 1.0000
Epoch 615/50000
13200/13200 [==============================] - 2s 178us/step - loss: 3.5765e-07 - accuracy: 1.0000 - val_loss: 3.5792e-07 - val_accuracy: 1.0000
Epoch 616/50000
13200/13200 [==============================] - 2s 175us/step - loss: 3.5765e-07 - accuracy: 1.0000 - val_loss: 3.5795e-07 - val_accuracy: 1.0000
Epoch 617/50000
13200/13200 [==============================] - 2s 151us/step - loss: 3.5765e-07 - accuracy: 1.0000 - val_loss: 3.5795e-07 - val_accuracy: 1.0000
Epoch 618/50000
13200/13200 [==============================] - 2s 169us/step - loss: 3.5765e-07 - accuracy: 1.0000 - val_loss: 3.5795e-07 - val_accuracy: 1.0000
Epoch 619/50000
13200/13200 [==============================] - 2s 159us/step - loss: 3.5766e-07 - accuracy: 1.0000 - val_loss: 3.5803e-07 - val_accuracy: 1.0000
Epoch 620/50000
13200/13200 [=====

Epoch 665/50000
13200/13200 [==============================] - 2s 157us/step - loss: 3.5765e-07 - accuracy: 1.0000 - val_loss: 3.5799e-07 - val_accuracy: 1.0000
Epoch 666/50000
13200/13200 [==============================] - 2s 151us/step - loss: 3.5764e-07 - accuracy: 1.0000 - val_loss: 3.5799e-07 - val_accuracy: 1.0000
Epoch 667/50000
13200/13200 [==============================] - 2s 147us/step - loss: 3.5765e-07 - accuracy: 1.0000 - val_loss: 3.5799e-07 - val_accuracy: 1.0000
Epoch 668/50000
13200/13200 [==============================] - 2s 176us/step - loss: 3.5764e-07 - accuracy: 1.0000 - val_loss: 3.5795e-07 - val_accuracy: 1.0000
Epoch 669/50000
13200/13200 [==============================] - 2s 158us/step - loss: 3.5765e-07 - accuracy: 1.0000 - val_loss: 3.5795e-07 - val_accuracy: 1.0000
Epoch 670/50000
13200/13200 [==============================] - 2s 155us/step - loss: 3.5764e-07 - accuracy: 1.0000 - val_loss: 3.5795e-07 - val_accuracy: 1.0000
Epoch 671/50000
13200/13200 [=====

Epoch 716/50000
13200/13200 [==============================] - 2s 153us/step - loss: 3.5764e-07 - accuracy: 1.0000 - val_loss: 3.5788e-07 - val_accuracy: 1.0000
Epoch 717/50000
13200/13200 [==============================] - 2s 171us/step - loss: 3.5764e-07 - accuracy: 1.0000 - val_loss: 3.5784e-07 - val_accuracy: 1.0000
Epoch 718/50000
13200/13200 [==============================] - 2s 147us/step - loss: 3.5765e-07 - accuracy: 1.0000 - val_loss: 3.5792e-07 - val_accuracy: 1.0000
Epoch 719/50000
13200/13200 [==============================] - 2s 151us/step - loss: 3.5764e-07 - accuracy: 1.0000 - val_loss: 3.5784e-07 - val_accuracy: 1.0000
Epoch 720/50000
13200/13200 [==============================] - 2s 159us/step - loss: 3.5765e-07 - accuracy: 1.0000 - val_loss: 3.5792e-07 - val_accuracy: 1.0000
Epoch 721/50000
13200/13200 [==============================] - 2s 158us/step - loss: 3.5764e-07 - accuracy: 1.0000 - val_loss: 3.5795e-07 - val_accuracy: 1.0000
Epoch 722/50000
13200/13200 [=====

In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

In [14]:
print(f'The score (accurary) in the training set is {model.evaluate(X_train,t_train)[1]*100:.02f}%')
print(f'The score (accuracy) in the test set is {model.evaluate(X_test,t_test)[1]*100:.02f}%\n')

from sklearn.metrics import confusion_matrix

y_test = model.predict_classes(X_test)
conf_mx = confusion_matrix(list(map(int,t_test)), y_test)
print(conf_mx)

57200/57200 [==============================] - 3s 56us/step
The score (accurary) in the training set is 38.08%
14300/14300 [==============================] - 1s 58us/step
The score (accuracy) in the test set is 36.29%

[[1161  135    1   17    3    4    0    3    0    0    0]
 [ 890  220   21  160    6   35    0    6    0    0    0]
 [ 111   55  415  296    8  119  148  145   46    0    8]
 [ 285   91  264  360   13  131   44  110   12    0    4]
 [ 232   75  288  347   16  141   42  122   13    0    0]
 [ 127   69  301  262   11  179  114  162   12    0    1]
 [  26    5  294   77    2   53  341  137  301   12   34]
 [  74   24  192  142    4  103  106  547   84    1   45]
 [   1    0   54    7    1    9  175   83  508  214  208]
 [   0    0    2    0    0    0   11   15  209  805  228]
 [   0    0   20    1    0    7   17   77  191  375  637]]


In [135]:
print(t_test)

[ 6  9  8 ...  4 10  2]


In [16]:
# Cargamos dataset de ajuste
index_ajuste = pandas.read_csv("aleatorio-dataset1/index.txt")
index_ajuste["numframes"] = index_ajuste.framelst-index_ajuste.frameini+1
maxframes_ajuste = index_ajuste.numframes.max()
print(maxframes_ajuste)
frames_ajuste = pandas.read_csv("aleatorio-dataset1/frames.txt")
frames_ajuste.head(-1)

33


,tagsread,pwread,pwcol,pwmin,pwmax
0,1,1.566386,0.000000,1.566386,1.566386
1,1,1.420922,0.000000,1.420922,1.420922
2,5,0.127248,0.135332,0.104700,0.135545
3,4,0.125929,0.000000,0.107457,0.138935
4,1,0.687071,0.000000,0.687071,0.687071
...,...,...,...,...,...
49206,0,0.000000,0.000000,0.000000,0.000000
49207,0,0.000000,0.000000,0.000000,0.000000
49208,0,0.000000,0.000000,0.000000,0.000000
49209,0,0.000000,0.000000,0.000000,0.000000


In [17]:
SAMPLES_BATCH = 200
NUM_BATCHES = 100

XX = np.zeros(shape=[SAMPLES_BATCH, 23]) 
tt = np.zeros(shape=[SAMPLES_BATCH, ]) 
print(f'XX.shape: {XX.shape}')

mejora = 0.0
mejora_independiente = 0.0

for batch in range(NUM_BATCHES):
    # batch = random.randint(0,100) 
    for i in range(SAMPLES_BATCH):
        tagsread = index_ajuste.iloc[batch+i].tagsread
        readtime = index_ajuste.iloc[batch+i].readtime
        numframes = index_ajuste.iloc[batch+i].numframes
        framebase = int(index_ajuste.iloc[batch+i].frameini) 
        ha = index_ajuste.iloc[batch+i].ha
        hb = index_ajuste.iloc[batch+i].hb
        d = index_ajuste.iloc[batch+i].d
        aux = frames_ajuste.loc[framebase:framebase+numframes-1].values
        XX[i,0] = tagsread
        XX[i,1] = readtime
        XX[i,2] = numframes
        XX[i,3:8] = aux.mean(axis=0)
        XX[i,8:13] = aux.std(axis=0)
        XX[i,13:18] = aux.min(axis=0)
        XX[i,18:23] = aux.max(axis=0)
        tt[i] = index_ajuste.iloc[batch+i].boxoriginal
    #    if (i+1)%50==0:
    #        print(f'i: {i+1}, batch: {batch}, frames: {numframes}, framebase: {framebase}, tt: {tt[i]}, aux: {aux}, ha: {ha}, hb: {hb}, d: {d}')

    # Vemos resultado con la red anterior sin ajustar. Debe ser malo por no tener ningun dato de esta red
    XX_transform = preprocessor.transform(XX)
    precision_modelo_original = model.evaluate(XX_transform,tt)[1]
    print(f'precision_modelo_original: {precision_modelo_original*100:.02f}%')

    # Probamos usando solo dataset de ajuste. Separamos el conjunto de ajuste en train (100) y test (100)
    XX_train, XX_test, tt_train, tt_test =  train_test_split(XX, tt, test_size=0.5)
    preprocessor_ajuste = ColumnTransformer(
            remainder='passthrough', #passthough features not listed
            transformers=[
                ('std', standard_transformer , []),#slice(0,X.shape[1])),
                ('mm', minmax_transformer , slice(0,X.shape[1]))
            ])
    XX_train = preprocessor_ajuste.fit_transform(XX_train)
    XX_test = preprocessor_ajuste.transform(XX_test)
    model_ajuste = keras.models.Sequential([
        keras.layers.Dense(100, activation="relu", input_shape=X_train.shape[1:]),
        keras.layers.Dense(50, activation="tanh"),
        keras.layers.Dense(int(t.max())+1, activation="sigmoid")
    ])
    model_ajuste.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    model_ajuste.fit(XX_train, tt_train, epochs=50000,  validation_data=(XX_test, tt_test),  batch_size=32, callbacks=[early_stopping_cb], verbose=0)
    precision_modelo_independiente = model_ajuste.evaluate(XX_test,tt_test)[1]
    print(f'precision_modelo_independiente: {precision_modelo_independiente*100:.02f}%')

    # Ajuste mejorado del modelo original con transfer learning
    XX_train, XX_test, tt_train, tt_test =  train_test_split(preprocessor.transform(XX), tt, test_size=0.5)
    model_copy= keras.models.clone_model(model)
    model_copy.build((None, XX.shape[1])) # replace 10 with number of variables in input layer
    model_copy.set_weights(model.get_weights())
    model_copy.layers[0].trainable = False
    model_copy.layers[1].trainable = False
    model_copy.layers[2].trainable = False
    #for l in model_copy.layers:
    #    print(l.name, l.trainable)
    model_copy.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=["accuracy"])

    model_copy.fit(XX_train, tt_train, epochs=5000,  validation_data=(XX_test, tt_test),  batch_size=32, callbacks=[early_stopping_cb], verbose=0)
    precision_modelo_copia = model_copy.evaluate(XX_test,tt_test)[1]
    print(f'precision_modelo_copia: {precision_modelo_copia*100:.02f}%')

    mejora += (precision_modelo_copia - precision_modelo_original)/NUM_BATCHES
    mejora_independiente += (precision_modelo_copia - precision_modelo_independiente)/NUM_BATCHES
    
    
print(f'Mejora: {mejora*100:.02f}%')
print(f'Mejora independiente: {mejora_independiente*100:.02f}%')

XX.shape: (200, 23)
200/200 [==============================] - 0s 68us/step
precision_modelo_original: 24.50%
100/100 [==============================] - 0s 89us/step
precision_modelo_independiente: 25.00%
100/100 [==============================] - 0s 92us/step
precision_modelo_copia: 33.00%
200/200 [==============================] - 0s 69us/step
precision_modelo_original: 24.50%
100/100 [==============================] - 0s 90us/step
precision_modelo_independiente: 22.00%
100/100 [==============================] - 0s 90us/step
precision_modelo_copia: 23.00%
200/200 [==============================] - 0s 73us/step
precision_modelo_original: 24.50%
100/100 [==============================] - 0s 87us/step
precision_modelo_independiente: 21.00%
100/100 [==============================] - 0s 88us/step
precision_modelo_copia: 25.00%
200/200 [==============================] - 0s 60us/step
precision_modelo_original: 24.50%
100/100 [==============================] - 0s 84us/step
precision_modelo_i

/Users/javiervalesalonso/opt/anaconda3/envs/ml/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.102035). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


100/100 [==============================] - 0s 94us/step
precision_modelo_copia: 31.00%
200/200 [==============================] - 0s 76us/step
precision_modelo_original: 23.50%
100/100 [==============================] - 0s 88us/step
precision_modelo_independiente: 18.00%
100/100 [==============================] - 0s 89us/step
precision_modelo_copia: 23.00%
200/200 [==============================] - 0s 62us/step
precision_modelo_original: 23.50%
100/100 [==============================] - 0s 80us/step
precision_modelo_independiente: 26.00%
100/100 [==============================] - 0s 91us/step
precision_modelo_copia: 29.00%
200/200 [==============================] - 0s 74us/step
precision_modelo_original: 24.00%
100/100 [==============================] - 0s 79us/step
precision_modelo_independiente: 24.00%
100/100 [==============================] - 0s 87us/step
precision_modelo_copia: 25.00%
200/200 [==============================] - 0s 57us/step
precision_modelo_original: 24.00%
100/100

100/100 [==============================] - 0s 112us/step
precision_modelo_copia: 30.00%
200/200 [==============================] - 0s 67us/step
precision_modelo_original: 28.00%
100/100 [==============================] - 0s 110us/step
precision_modelo_independiente: 20.00%
100/100 [==============================] - 0s 89us/step
precision_modelo_copia: 30.00%
200/200 [==============================] - 0s 105us/step
precision_modelo_original: 27.50%
100/100 [==============================] - 0s 71us/step
precision_modelo_independiente: 27.00%
100/100 [==============================] - 0s 87us/step
precision_modelo_copia: 27.00%
200/200 [==============================] - 0s 69us/step
precision_modelo_original: 27.50%
100/100 [==============================] - 0s 77us/step
precision_modelo_independiente: 19.00%
100/100 [==============================] - 0s 100us/step
precision_modelo_copia: 24.00%
200/200 [==============================] - 0s 88us/step
precision_modelo_original: 27.50%
100

In [21]:
model_copy.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               2400      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_4 (Dense)              (None, 11)                561       
Total params: 18,111
Trainable params: 561
Non-trainable params: 17,550
_________________________________________________________________


In [22]:
t.min()

0.0

In [23]:
t.max()

10.0

In [30]:
print(X_test[0])

[0.         0.02045572 0.         0.06896552 0.49301994 0.
 0.51170601 0.43111907 0.         0.         0.         0.
 0.         0.07692308 0.49320817 0.         0.51184197 0.43131815
 0.         0.46442789 0.         0.47237981 0.40220314]


In [41]:
model.predict(X0[:10])

array([[8.5861194e-01, 9.9990475e-01, 0.0000000e+00, 5.4538250e-06,
        4.7683716e-07, 1.8596649e-05, 5.9604645e-08, 1.7881393e-07,
        2.0861626e-07, 0.0000000e+00, 2.2351742e-06],
       [6.6755629e-01, 9.9999887e-01, 5.6624413e-07, 2.9802322e-08,
        6.8545341e-06, 5.6624413e-07, 8.9406967e-08, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 2.6822090e-07],
       [8.5864538e-01, 9.9990463e-01, 0.0000000e+00, 5.3942204e-06,
        4.1723251e-07, 1.8537045e-05, 8.9406967e-08, 2.3841858e-07,
        2.0861626e-07, 0.0000000e+00, 2.1457672e-06],
       [8.5863322e-01, 9.9990469e-01, 0.0000000e+00, 5.3942204e-06,
        4.1723251e-07, 1.8566847e-05, 8.9406967e-08, 2.0861626e-07,
        2.0861626e-07, 0.0000000e+00, 2.2649765e-06],
       [8.5855758e-01, 9.9990487e-01, 0.0000000e+00, 5.3942204e-06,
        3.8743019e-07, 1.8566847e-05, 1.4901161e-07, 2.6822090e-07,
        2.0861626e-07, 0.0000000e+00, 2.2351742e-06],
       [7.9128516e-01, 9.9998528e-01, 8.9406967e-0

In [35]:
print(t_test[2])

0.0


In [38]:
X0 = X_test[t_test==0]

In [39]:
print(X0.shape)

(1366, 23)


In [55]:
index.head()

,frameini,framelst,boxnum,boxoriginal,L,W,H,read,tags,tagsread,readtime,despX,despY,theta,vhr,ha,hb,d,numframes
0,0,1,8,8,0.590,0.390,0.400,1,10,10,0.03092,0.129186,0.096514,13.412621,0,1.55,0.55,3.3,2
1,2,4,26,9,0.400,0.585,0.600,1,15,15,0.03983,0.040972,0.160102,21.612419,2,1.55,0.55,3.3,3
2,5,6,22,7,0.131,0.587,0.581,1,5,5,0.01907,-0.093494,0.158079,-14.490815,0,1.55,0.55,3.3,2
3,7,9,27,9,0.600,0.585,0.400,1,15,15,0.02296,0.134029,-0.121634,-30.319941,1,1.55,0.55,3.3,3
4,10,10,4,4,0.386,0.260,0.266,1,3,3,0.01072,0.108996,0.058330,11.509462,2,1.55,0.55,3.3,1


In [59]:
index[index.boxoriginal==0].shape

(487038, 19)

In [61]:
index[index.boxoriginal==10].shape

(729315, 19)

In [18]:
for i in range(t.max()):
    print(f'Tags en cajas tipo {i}: {index[index.boxoriginal==i].shape[0]/index.shape[0]*100:.2f}')
    print(f'Cajas tipo {i}: {X[t==i].shape[0]/X.shape[0]*100:.2f}')

Tags en cajas tipo 0: 6.66
Cajas tipo 0: 9.09
Tags en cajas tipo 1: 10.02
Cajas tipo 1: 9.09
Tags en cajas tipo 2: 9.99
Cajas tipo 2: 9.09
Tags en cajas tipo 3: 10.00
Cajas tipo 3: 9.09
Tags en cajas tipo 4: 9.99
Cajas tipo 4: 9.09
Tags en cajas tipo 5: 6.66
Cajas tipo 5: 9.09
Tags en cajas tipo 6: 6.66
Cajas tipo 6: 9.09
Tags en cajas tipo 7: 9.98
Cajas tipo 7: 9.09
Tags en cajas tipo 8: 10.01
Cajas tipo 8: 9.09
Tags en cajas tipo 9: 10.01
Cajas tipo 9: 9.09


In [149]:
for i in range(t.max()):
    print(f'Tags en cajas tipo {i}: {index[index.boxoriginal==i].tagsread[:100]}')

Tags en cajas tipo 0: 5       1
31      1
33      1
51      1
76      1
       ..
1355    1
1360    1
1362    1
1363    1
1381    1
Name: tagsread, Length: 100, dtype: int64
Tags en cajas tipo 1: 17     1
30     1
35     1
36     1
59     1
      ..
957    1
969    1
971    1
974    1
990    1
Name: tagsread, Length: 100, dtype: int64
Tags en cajas tipo 2: 9       4
22      4
48      4
49      4
52      4
       ..
1029    4
1043    4
1058    4
1099    4
1103    4
Name: tagsread, Length: 100, dtype: int64
Tags en cajas tipo 3: 15     3
19     3
23     3
26     3
27     3
      ..
921    3
923    3
930    3
963    3
965    3
Name: tagsread, Length: 100, dtype: int64
Tags en cajas tipo 4: 4      3
7      3
21     3
32     3
39     3
      ..
948    3
949    3
955    3
970    3
985    3
Name: tagsread, Length: 100, dtype: int64
Tags en cajas tipo 5: 50      4
71      4
73      4
80      4
103     4
       ..
1392    4
1407    4
1411    4
1414    4
1419    4
Name: tagsread, Length: 100, dt

In [150]:
index[index.boxoriginal==i].head(50)

,frameini,framelst,boxnum,boxoriginal,L,W,H,read,tags,tagsread,readtime,despX,despY,theta,vhr,ha,hb,d,numframes
1,2,4,26,9,0.4,0.585,0.600,1,15,15,0.03983,0.040972,0.160102,21.612419,2,1.55,0.55,3.3,3
3,7,9,27,9,0.6,0.585,0.400,1,15,15,0.02296,0.134029,-0.121634,-30.319941,1,1.55,0.55,3.3,3
6,12,13,9,9,0.6,0.400,0.585,1,15,15,0.04277,0.054632,-0.190784,-28.103867,0,1.55,0.55,3.3,2
8,15,16,26,9,0.4,0.585,0.600,1,15,15,0.04277,0.006993,0.043028,36.409745,2,1.55,0.55,3.3,2
11,27,28,9,9,0.6,0.400,0.585,1,15,15,0.04068,-0.164511,-0.032152,-16.813992,2,1.55,0.55,3.3,2
25,49,51,27,9,0.6,0.585,0.400,1,15,15,0.04429,0.059907,-0.114639,12.099951,2,1.55,0.55,3.3,3
29,55,57,9,9,0.6,0.400,0.585,1,15,15,0.04666,0.147422,0.001961,-25.025301,0,1.55,0.55,3.3,3
43,76,79,27,9,0.6,0.585,0.400,1,15,15,0.04999,0.126045,0.138963,-35.906644,2,1.55,0.55,3.3,4
47,84,85,9,9,0.6,0.400,0.585,1,15,15,0.04277,-0.115452,-0.135786,-24.005346,0,1.55,0.55,3.3,2
66,111,121,26,9,0.4,0.585,0.600,0,15,14,0.07052,0.117095,-0.008755,41.212407,2,1.55,0.55,3.3,11
